In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
#list the current working dir
os.getcwd()
#change the current working dir
os.chdir('/content')


In [ ]:
!unzip /content/drive/MyDrive/Tomat/tomat_train.zip -d /content/tomat/

Archive:  /content/drive/MyDrive/Tomat/tomat_train.zip
   creating: /content/tomat/tomat_train/test/
   creating: /content/tomat/tomat_train/test/Ripe/
  inflating: /content/tomat/tomat_train/test/Ripe/r (1464).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1484).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1501).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1533).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1556).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1570).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1589).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1608).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1611).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1613).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1623).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1627).jpg  
  inflating: /content/tomat/tomat_train/test/Ripe/r (1636).jpg  
  i

In [ ]:
!mkdir /content/saved_model
!cp /content/drive/MyDrive/Tomat/saved_model/* /content/saved_model
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12
!pip uninstall cupy-cuda12x cupy-cuda11x -y
!pip install cupy-cuda11x

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.2/955.2 MB 733.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.4/582.4 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.svm import SVC
from cuml import SVC as cuSVC
from sklearn.model_selection import train_test_split
import cuml.model_selection
from cuml.metrics import accuracy_score
import numpy as np
import joblib
from tensorflow.keras.models import save_model
# from sklearn.model_selection import GridSearchCV

# Define the input shape based on the VGG16 model requirements
input_shape = (224, 224, 3)
num_classes = 2  # Number of classes (ripe and unripe)
learning_rate = 1e-3
batch_size = 16
total_epochs = 100
early_stopping_patience = 20
dropout_rate = 0.3


In [ ]:

# Load pre-trained VGG16 model without top classification layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

## Unfreeze the last two layers of the base model for fine-tuning
# Freeze all layers for extracting features
for layer in base_model.layers:
    layer.trainable = False

# base_model.compile(optimizer=SGD(lr=learning_rate),
#                             loss='binary_crossentropy',
#                             metrics=['accuracy'])
# # Early stoppage
# early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping_patience, verbose=1, restore_best_weights=True)

base_model.summary()



58889256/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:

# Define paths to your dataset (replace these with your actual dataset paths)
train_data_dir = r'/content/tomat/tomat_train/train'
validation_data_dir = r'/content/tomat/tomat_train/validation'
test_data_dir = r'/content/tomat/tomat_train/test'


# Data augmentation for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    width_shift_range=0.5,
    height_shift_range=0.5,
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.1, 0.7]
)

# Data augmentation for the validation and test datasets
validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary'
)



Found 3559 images belonging to 2 classes.
Found 195 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [ ]:
# Extract features from the VGG16 base model
def extract_features(generator, model):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features) * batch_size >= generator.samples:
            break

    # Ensure all batches have the same shape
    features = [batch.reshape((batch.shape[0], -1)) for batch in features]

    return np.vstack(features), np.concatenate(labels)

# Extract features from the training set
train_features, train_labels = extract_features(train_generator, base_model)

# Extract features from the validation set (if needed)
validation_features, validation_labels = extract_features(validation_generator, base_model)

# Extract features from the test set
test_features, test_labels = extract_features(test_generator, base_model)

# Join train and validation features as cross validation is performed
train_and_val_features, train_and_val_labels= np.vstack([train_features, validation_features]), np.concatenate([train_labels, validation_labels])



1/1 [==============================] - 1s 1s/step


In [ ]:

# X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(train_and_val_features, train_and_val_labels, test_size=0.2, random_state=42)

# Define the parameter grid
param_grid = {'C': [515, 516, 517], 'gamma': ['auto', 'scale']}
param_grid2 = {'coef0' : [2, 2.2, 2.4, 2.6, 3], 'gamma' : ['auto', 'scale'], 'kernel': ['poly'], 'degree': [2, 3, 4]}
param_grid3 = {'coef0' : [i/10 for i in range(-10, -20, -1)], 'gamma' : ['auto', 'scale'], 'kernel': ['sigmoid']}

best_accuracy = 0.0
best_params = {}
best_model = None  # Variable to store the best model

# Perform grid search for four kernels

for gamma in param_grid3['gamma']:
  for coef0 in param_grid3['coef0']:
    # initialize cumL SVM classifier
        cuml_svm_classifier = cuSVC(kernel='sigmoid', coef0=coef0, gamma=gamma)

        # Train the cuML SVM classifier on the training set
        cuml_svm_classifier.fit(X_train, y_train)

        # Make predictions using the trained cuML SVM on the validation set
        cuml_svm_predictions = cuml_svm_classifier.predict(X_val)

        # Calculate accuracy on the validation set
        accuracy = accuracy_score(y_val, cuml_svm_predictions)

        # Check if the current model is the best
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'coef0' : coef0, 'gamma' : gamma}

            best_model = cuml_svm_classifier

for degree in param_grid2['degree']:
  for gamma in param_grid2['gamma']:
    for coef0 in param_grid2['coef0']:
      # initialize cumL SVM classifier
        cuml_svm_classifier = cuSVC(kernel='poly', degree=degree, coef0=coef0, gamma=gamma)

        # Train the cuML SVM classifier on the training set
        cuml_svm_classifier.fit(X_train, y_train)

        # Make predictions using the trained cuML SVM on the validation set
        cuml_svm_predictions = cuml_svm_classifier.predict(X_val)

        # Calculate accuracy on the validation set
        accuracy = accuracy_score(y_val, cuml_svm_predictions)

        # Check if the current model is the best
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'coef0' : coef0, 'gamma' : gamma, 'degree': degree}

            best_model = cuml_svm_classifier


for C in param_grid['C']:
   # Initialize cuML SVM classifier
        cuml_svm_classifier = cuSVC(C=C, kernel='linear')

        # Train the cuML SVM classifier on the training set
        cuml_svm_classifier.fit(X_train, y_train)

        # Make predictions using the trained cuML SVM on the validation set
        cuml_svm_predictions = cuml_svm_classifier.predict(X_val)

        # Calculate accuracy on the validation set
        accuracy = accuracy_score(y_val, cuml_svm_predictions)

        # Check if the current model is the best
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'C': C}
            best_model = cuml_svm_classifier

for C in param_grid['C']:
    for gamma in param_grid['gamma']:
        # Initialize cuML SVM classifier
        cuml_svm_classifier = cuSVC(C=C, kernel='rbf', gamma=gamma)

        # Train the cuML SVM classifier on the training set
        cuml_svm_classifier.fit(X_train, y_train)

        # Make predictions using the trained cuML SVM on the validation set
        cuml_svm_predictions = cuml_svm_classifier.predict(X_val)

        # Calculate accuracy on the validation set
        accuracy = accuracy_score(y_val, cuml_svm_predictions)

        # Check if the current model is the best
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'C': C, 'gamma': gamma}
            best_model = cuml_svm_classifier

print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)


Best Parameters: {'C': 515, 'gamma': 'scale'}
Best Accuracy: 0.8548601865768433


In [ ]:
# Save the VGG16 base model
base_model.save('/content/Tomat/vgg16_base_model.h5')

# # Save the best model
# best_model.save("/content/Tomat/cuml_svm_model")

# Save the SVM classifier
joblib.dump(best_model, '/content/Tomat/cuml_svm_model.joblib')

# If you want to save the SVM classifier with its probability information
# (needed for decision_function and predict_proba methods)
joblib.dump(best_model, '/content/Tomat/cuml_svm_model_prob.joblib', compress=True)

# Save the labels (optional)
np.save('/content/Tomat/labels.npy', train_generator.class_indices)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:

# Make predictions using the trained SVM
svm_predictions = best_model.predict(test_features)

# Print the accuracy achieved by the best model on the validation set
best_model_test_accuracy = best_model.score(test_features, test_labels)
print(f'Best Model Test Accuracy: {best_model_test_accuracy * 100:.2f}%')

Best Model Test Accuracy: 83.08%


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# !zip -r saved_model2.zip /content/Tomat/
# from google.colab import files
# files.download('saved_model2.zip')
# !mkdir /content/drive/MyDrive/Tomat/saved_model2
# !cp /content/Tomat/* -d /content/drive/MyDrive/Tomat/saved_model2/